### Data Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


# reading the csv file using pandas 

data = pd.read_csv('garments_worker_productivity.csv')


In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
# checking for missing values
data.wip.isnull().sum()

In [ ]:
# checking for duplicates
data.duplicated().sum()

### Data Cleaning

In [ ]:
# removing the missing data from wip 

dropped_data = data.drop(labels=["wip"], axis=1, inplace=False)

In [ ]:
dropped_data

In [ ]:
dropped_data.quarter.unique()

In [ ]:
dropped_data.department.unique()

In [ ]:
dropped_data.day.unique()

### Handling Datetime Column

In [ ]:
df = pd.to_datetime(dropped_data["date"])

In [ ]:
dropped_data["year"] = df.dt.year

In [ ]:
dropped_data["month"] = df.dt.month

In [ ]:
dropped_data["day"] = df.dt.day

In [ ]:
dropped_data["weekday"] = df.dt.weekday

In [ ]:
dropped_data.drop(labels=["date"], axis=1, inplace=True)

In [ ]:
dropped_data.info()

In [ ]:
dropped_data.head()

### Sorting the Wrong Spelling Issue.

In [ ]:
dropped_data['department'] = dropped_data['department'].replace('sweing', 'sewing')
dropped_data['department'] = dropped_data['department'].replace('finishing ', 'finishing')

In [ ]:
dropped_data.department.unique()

### Transforming Actual Productivity to two classes

In [ ]:
dropped_data['actual_productivity_class'] = np.where(dropped_data['actual_productivity'] >= 0.80, 'Satifactory', 'Non-Satisfactory')


In [ ]:
dropped_data.department.value_counts()

In [ ]:
dropped_data.head()

### Visualization of Data

In [ ]:
# Plotting a bar chart of the actual productivity by department
plt.figure(figsize=(10, 6))
sns.barplot(x='department', y='actual_productivity', data=dropped_data)
plt.title('Actual Productivity by Department')
plt.xlabel('Department')
plt.ylabel('Actual Productivity')
plt.show()


In [ ]:
sns.barplot(x='department', y='targeted_productivity', data=dropped_data)
plt.title('Targeted Productivity by Department')
plt.xlabel('Department')
plt.ylabel('Targeted Productivity')
plt.show()


### Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import SGD

In [ ]:
encoder = LabelEncoder()

In [ ]:
# checking for categorical columns
obj_col = list(dropped_data.select_dtypes(include='object').columns)

In [ ]:
obj_col

In [ ]:
dropped_data[obj_col] = dropped_data[obj_col].apply(encoder.fit_transform)

In [ ]:
dropped_data.head()

In [ ]:
dropped_data.info()

In [ ]:
norm = Normalizer()

In [ ]:
X =  dropped_data.drop(['actual_productivity_class'], axis=1)
y = dropped_data["actual_productivity_class"]

In [ ]:
X = norm.fit_transform(X)

In [ ]:
train, test = train_test_split(dropped_data, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=.2, random_state=42)

In [ ]:
y_test.shape

### Model Training

#### Model Building with Scratch ANN

In [ ]:
from activations import back_propagation, evaluate_algorithm

In [ ]:
from model_new import NeuralNetwork, vectorized_result

In [ ]:
y_train = [vectorized_result(i) for i in y_train]

In [ ]:
n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y_train))
n_neurons = 20


network = NeuralNetwork([n_inputs, n_neurons, n_neurons])
train_data = list(zip(X_train, y_train))
network.fit(train_data, 30, 10, 3.0)

In [ ]:
len(np.unique(y_train))

In [ ]:

network = NeuralNetwork([16, 20, 2])
train_data = list(zip(X_train, y_train))
network.fit(train_data, 30, 5, 3.0)

#### Model Building with Keras Model

In [ ]:
model_keras = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model_keras.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model_keras.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


In [ ]:
# Evaluate the model
loss, accuracy = model_keras.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


##### Model tuning

In [ ]:
from keras.layers import Dropout, BatchNormalization
from keras import regularizers

In [ ]:
model_tune = Sequential()
model_tune.add(Dense(units=20, activation='relu', input_dim=X_train.shape[1]))
model_tune.add(Dropout(0.5))
model_tune.add(Dense(64, input_dim=64,
                kernel_regularizer=regularizers.l1(0.01),
                activity_regularizer=regularizers.l2(0.01)))
model_tune.add(BatchNormalization())
model_tune.add(Dense(units=1, activation='softmax'))

In [ ]:
model_tune.compile(optimizer=SGD(learning_rate=0.1), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_tune.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

#### Model Building with RandomForest

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
rf.fit(X_train, y_train)


In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
f1_score(y_test, y_pred)

In [ ]:
accuracy_score(y_test, y_pred)